# SPARQL in Ruby scripts

Executing a SPARQL query in Ruby is extremely simple. It uses the [SPARQL client](http://www.rubydoc.info/github/ruby-rdf/sparql-client/frames) Ruby Gem, which uses [RDF::Query::Solutions](http://www.rubydoc.info/github/ruby-rdf/rdf/RDF/Query/Solutions)and the code is as follows:



In [2]:

require 'sparql/client'

endpoint = "http://sparql.uniprot.org/sparql"
query = 
<<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>
SELECT ?name ?image
WHERE 
{
  ?taxon  <http://xmlns.com/foaf/0.1/depiction>  ?image .
  ?taxon <http://purl.uniprot.org/core/scientificName> ?name .
  FILTER regex(?name, '^Arabidopsis.*', 'i') .
}
END

sparql = SPARQL::Client.new(endpoint)
result = sparql.query(query)

puts result.class
result.each do |solution|
  puts "Species #{solution[:name]} is depicted by image #{solution[:image]}"
end
puts ""

RDF::Query::Solutions
Species Arabidopsis lyrata subsp. petraea is depicted by image https://upload.wikimedia.org/wikipedia/commons/2/25/Northern_Rock-cress_%28Cardaminopsis_petraea%29_-_geograph.org.uk_-_723142.jpg
Species Arabidopsis thaliana is depicted by image https://upload.wikimedia.org/wikipedia/commons/3/39/Arabidopsis.jpg
Species Arabidopsis thaliana is depicted by image https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/Arabidopsis_thaliana_inflorescencias.jpg/800px-Arabidopsis_thaliana_inflorescencias.jpg
Species Arabidopsis arenosa is depicted by image https://upload.wikimedia.org/wikipedia/commons/2/22/Cardaminopsis_arenosa.jpeg
Species Arabidopsis arenosa is depicted by image https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Cardaminopsis_arenosa_habitus.jpeg/450px-Cardaminopsis_arenosa_habitus.jpeg




### and that's really all there is to it!  Easy!

# Loading, Creating and Writing RDF in Ruby

Here we will use the [<code>rdf</code> ruby gem](http://ruby-rdf.github.io/rdf/) which is extremely easy to understand.  There are two ways to work with RDF in Ruby: 

### Method 1
You can start with an RDF file (either locally or on the Web), and load it into a "graph" as follows:



In [3]:
require 'rdf'
require 'rdf/ntriples'

graph = RDF::Graph.load("http://ruby-rdf.github.com/rdf/etc/doap.nt")
puts graph.class
puts graph.size   # number of triples in the graph

RDF::Graph
85



(note that we will ONLY use ntriples in this course - a more common RDF format is 'Turtle' but there are efficiencies we can gain by using ntriples format, so that is what we will use.  You can read the documentation for yourself to learn how to load other RDF formats...)

Once we have loaded the data, we convert it into an RDF::Repository Object, and then use that exactly as we used the SPARQL endpoint in the example above:
 

In [4]:
require 'rdf'
require 'sparql/client'
require 'rdf/ntriples'
require 'rdf/repository'

graph = RDF::Graph.load("http://ruby-rdf.github.com/rdf/etc/doap.nt")

repo = RDF::Repository.new   # a repository can be used as a SPARQL endpoint for SPARQL::Client
repo.insert(*graph)   # the "splat" operator (*) converts an array into a list of individual things, in this case, the individual triples in the RDF::Graph
puts repo.count

query = 
<<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>
SELECT ?name
WHERE 
{
  ?p a foaf:Person .
  ?p foaf:name ?name .
}
END

sparql2 = SPARQL::Client.new(repo)
result = sparql2.query(query)

result.each do |solution|
  puts "Database contains the person:  #{solution[:name]}"  # call the pairs of variables in our query
end
puts ""

RDF::Graph
85
85
Database contains the person:  Pius Uzamere
Database contains the person:  Gregg Kellogg
Database contains the person:  Ben Lavender
Database contains the person:  Hellekin O. Wolf
Database contains the person:  Joey Geiger
Database contains the person:  Călin Ardelean
Database contains the person:  Keita Urashima
Database contains the person:  Arto Bendiken
Database contains the person:  Fumihiro Kato
Database contains the person:  John Fieber
Database contains the person:  Danny Gagne
Database contains the person:  Naoki Kawamukai




### Method 2
Another way is to create the RDF _de novo_.  For this, we will again use the rdf Gem, and in particular, the [RDF::Statement](http://www.rubydoc.info/github/ruby-rdf/rdf/master/RDF/Statement) Class.

It is useful to know some common ontologys for predicates:

| Ontology | URL |
| --- | --- |
| Dublin Core | http://purl.org/dc/terms/ |
| Friend of a Friend (FOAF) |  http://xmlns.com/foaf/0.1/knows |
| RDF | http://www.w3.org/1999/02/22-rdf-syntax-ns |
| RDF Schema (RDFS) | http://www.w3.org/2000/01/rdf-schema |
| Web Ontology Language (OWL) | http://www.w3.org/2002/07/owl# |
| XML Schema Datatypes (XSD) | http://www.w3.org/2001/XMLSchema# |

(note that you can use ANY of the terms in those ontologies... for example DC.author or FOAF.name)


Here is an example of the code:



In [5]:
require 'rdf'
require 'sparql/client'
require 'rdf/ntriples'
require 'rdf/repository'

repo = RDF::Repository.new   # a repository can be used as a SPARQL endpoint for SPARQL::Client

# create the vocabularies I will use
foaf = RDF::Vocabulary.new("http://xmlns.com/foaf/0.1/")
dc = RDF::Vocabulary.new("http://purl.org/dc/terms/")
rdf = RDF::Vocabulary.new("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

# create the Subject, Predicate, and Object of a triple
s = RDF::URI.new("http://wilkinsonlab.info/Courses")
p = dc.creator
o = RDF::URI.new("http://wilkinsonlab.info/#mark")
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository

# create the Subject, Predicate, and Object of the next triple
s = RDF::URI.new("http://wilkinsonlab.info/#mark")
p = foaf.name
o = RDF::Literal.new("Mark Wilkinson")
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository

# create the Subject, Predicate, and Object of the last triple
s = RDF::URI.new("http://wilkinsonlab.info/#mark")
p = rdf.type
o = foaf.Person
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository

# and from here, it is exactly the same as before...

query = 
<<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>
SELECT ?name
WHERE
{
  ?p a foaf:Person .
  ?p foaf:name ?name
}
END

sparql2 = SPARQL::Client.new(repo)
result = sparql2.query(query)

result.each do |solution|
  puts "Database contains the person:  #{solution[:name]}"  # call the pairs of variables in our query
end
puts "done"

Database contains the person:  Mark Wilkinson
done


# Writing RDF to a file

The final thing we need to do is export our beautiful RDF to a file so that it can be loaded into a database.  This is easy too:

**(look in your Lectures/files/ folder to see the output...)**



In [5]:
RDF::Writer.open("./files/MarkWilkinson.nt") { |writer| writer << repo }

#<RDF::NTriples::Writer:0x000000000331ed38 @options={:validate=>true, :logger=>#<IRuby::OStream:0x000000000239ace0 @session=#<IRuby::Session:0x000000000239b668 @sockets={:publish=>#<CZTop::Socket::PUB:0x238abc0 last_endpoint="tcp://127.0.0.1:44011">, :reply=>#<CZTop::Socket::ROUTER:0x2367c40 last_endpoint="tcp://127.0.0.1:44535">, :stdin=>#<CZTop::Socket::ROUTER:0x23a07e0 last_endpoint="tcp://127.0.0.1:54901">}, @session="5a578698-c2ff-4bce-8a1a-9e2f8cb51721", @hmac=54085e082ba32b52036dc98f4ec28ec1025b94a3f14357a5fe3dcb57af2169a6, @last_recvd_msg={:idents=>["c227ff01ae764e94aaff02ac6f9dd203"], :header=>{"msg_id"=>"63325c7f05104614bb11f435000b69bc", "username"=>"username", "session"=>"c227ff01ae764e94aaff02ac6f9dd203", "msg_type"=>"execute_request", "version"=>"5.2", "date"=>"2019-11-20T15:17:21.519192Z"}, :parent_header=>{}, :metadata=>{}, :content=>{"code"=>"RDF::Writer.open(\"./files/MarkWilkinson.nt\") { |writer| writer << repo }\n", "silent"=>false, "store_history"=>true, "user_exp


# THAT's ALL!
